使用kafka接收，使用这种方式更简单灵活性更好

In [1]:
class Conf:
    pass
conf = Conf()
conf.kafka_servers = "localhost:9092"
conf.kafka_topic = "user_list_click"
conf.hbase_table = "user_list_click"
conf.hive_db = "resys_one"
conf.hive_table = "user_list_click"
conf.__dict__

{'kafka_servers': 'localhost:9092',
 'kafka_topic': 'user_list_click',
 'hbase_table': 'user_list_click',
 'hive_db': 'resys_one',
 'hive_table': 'user_list_click'}

In [2]:
import time
import happybase
import json
conn = happybase.Connection('localhost', autoconnect=True)
conn.open()
table = conn.table(conf.hbase_table)
def parse_save_hbase(line):
    jobj = json.loads(line)
    table.put(row=jobj['id'],
              data={
                    "d:click":jobj['click'],
                    "d:clickTime":time.strftime('%Y-%m-%d-%H-%M-%S',time.localtime(jobj['clickTime']/1000)),
                    "d:list":",".join(jobj['list']),
                    "d:recid":jobj['recid'],
                    "d:userid":jobj['userid'],
              },
              timestamp=1,
              wal=False)

In [ ]:
import kafka
from kafka import KafkaConsumer
consumer = KafkaConsumer('user_list_click', bootstrap_servers=['localhost:9092'])
for msg,i in zip(consumer,range(1,1000)):
    recv = "%s:%d:%d: key=%s value=%s" % (msg.topic, msg.partition, msg.offset, msg.key, msg.value)
    parse_save_hbase(bytes.decode(msg.value))
    print(recv)
    if i %5==0:
        break
print("退出kafka连接")
consumer.close()

user_list_click:0:134: key=b'2021121458193900089-400039135' value=b'{"click":"400039135","clickTime":1639470479781,"id":"2021121458193900089-400039135","list":["400039135","410302787","410590524","410616400","460075096","461087263","470262695","470424986","470745270","470925586","470930435","471152864","471278143","471421737","471755645"],"recid":"2021121458193900089","userid":"0000000001"}'
user_list_click:0:135: key=b'2021121458193900089-410590524' value=b'{"click":"410590524","clickTime":1639470481996,"id":"2021121458193900089-410590524","list":["400039135","410302787","410590524","410616400","460075096","461087263","470262695","470424986","470745270","470925586","470930435","471152864","471278143","471421737","471755645"],"recid":"2021121458193900089","userid":"0000000001"}'
user_list_click:0:136: key=b'2021121458193900089-461087263' value=b'{"click":"461087263","clickTime":1639470483939,"id":"2021121458193900089-461087263","list":["400039135","410302787","410590524","410616400","4

In [ ]:
conn.close()